## mini csep
### 配置文件
* 默认的配置文件路径：$CENTERCODE/TestingCenterConfiguration，diy时注意配置文件命名规则  
* Generic XML-format Configuration Elements can appear in any of the XML format configuration files for the CSEP Factory-like processing modules  
--postProcessFactory:使用xml文件指定postprocess程序的信息（位置、module、module中的数据）[文档](http://northridge.usc.edu/trac/csep/wiki/PostProcessFactoryCommandOption)  
--geographicalRegion:Introducing New Region see source:trunk/src/SCECModels/NewZealand/generic/NZGeographicalRegions.init.xml [文档](http://northridge.usc.edu/trac/csep/wiki/GenericXMLElements)
    - collection area: 进行forecast的区域
    - test area:实际观测区域
    
### 模块
* GeographicalRegions.py:用来画forecast的空间分布,-h显示帮助 [文档](http://northridge.usc.edu/trac/csep/wiki/GeographicalRegions.py)  
* Forecast.py:用来画forecast的震级分布（参照正态分布），-h显示帮助 [文档](http://northridge.usc.edu/trac/csep/wiki/Forecast.py)  
* CSEPXMLGeneric.py：用来转换预测结果格式，ascii和xml [文档](http://northridge.usc.edu/trac/csep/wiki/CSEPXMLGeneric.py)
* EvaluationTest.py: 用来做forecast的检验 [文档](http://northridge.usc.edu/trac/csep/wiki/EvaluationTest.py)

### 定义计算区域的几个地方
* /usr/local/csep-16.10.0/src/SCECModels/SNZone/src/ETAS/template.poly (ETAS model使用)
* /usr/local/csep/src/SCECModels/SNZone/src/EEPAS/relmblocks.dat
* /usr/local/csep-16.10.0/data/templates/forecast.td.dat
* /usr/local/csep-16.10.0/data/templates/csep-forecast-template-M4.xml  用来做forecast
* /usr/local/csep-16.10.0/src/CENCModels/SNZone/data/areas/getCatalog_TestArea_SNZone.dat
* /usr/local/csep-16.10.0/src/CENCModels/SNZone/data/areas/getCatalog_CollectionArea_SNZone.dat 

* 注：/usr/local/csep-16.10.0/data/templates/csep-forecast-template-M4.xml中定义了每个cell的中心点的经纬度以及每个cell东西、南北方向的
跨度，而/usr/local/csep-16.10.0/data/templates/forecast.td.dat定义了每个cell的四个角点，需要保证左下角点+东西/南北方向跨度的一半==前一个
文件cell中点；同时震级也是同样，要保证后一个文件的震级+震级跨度的一半==前一个文件中的震级。
### 定义震级的地方
* /usr/local/csep/src/SCECModels/SNZone/src/EEPAS/relmmagsoneday.dat

## ~~ETASModel 修改部分~~（新版本不再需要修改）
#### ~~csepetas.r文件~~
* ~~cata.current <- matrix(scan(control.input[2], skip=0), ncol=15, byrow=T)中ncol改为10，以符合我们的zmap格式~~
* ~~cata.history <- cbind(1:nrow(cata.history.raw), cata.history.raw[,7:9],cata.history.day, 1,1,0)~~
~~改为 cata.history <- cbind(1:nrow(cata.history.raw), cata.history.raw[,8],cata.history.raw[,7],cata.history.raw[,9],cata.history.day, 1,1,0)
以符合lon lat mag的顺序~~

#### ~~prediction.r文件~~


## ETASModel使用说明
* 地震目录的检查：  
    1. 使用pre_compile_zmap.py预处理地震目录（填补震源深度空缺、给经纬度增加随机性、给震级增加随机性），输入格式为zmap
    2. 检查地震目录的质量，包括随机性、地震发生率突变等（见下图）。其中B图在横向不应有大的空缺，否则意味着集中爆发了某一震级的地震；C图分布应该均匀，否则说明随机性不好；D图跳跃之后缓慢变直，地震目录应该包含跳跃部分，否则应该以直线开始。![地震目录检查](pics/test.png)
* 配置文件control.etas.in  
主程序csepetas.r使用，自己手动编辑。
>  CN1.1  <span style="color:blue">#模型名称</span>  
>  nscata.zmap  <span style="color:blue">#地震目录文件名</span>  
>  3.7  <span style="color:blue">#拟合用最小震级</span>  
>  100  <span style="color:blue">#最大地震深度，和地震目录中单位相同km</span>  
>  2017-01-28  <span style="color:blue">#预测起始日期（starting.day）==拟合用地震目录的最后一天（包含，程序中挑选条件是<=starting.day）</span>  
>  2017-01-29  <span style="color:blue">#预测终止日期（ending.day），预测时间区间为ending.day-starting.day</span>  
>  1  <span style="color:blue">#是否优化，1/0</span>  
>  para  <span style="color:blue">#初始参数文件，只使用最后一行，计算结束会更新该文件</span>  
>  china.forecast.nodes.dat  <span style="color:blue">#预测的格点位置</span>  
>  0  <span style="color:blue">#没用？</span>  
>  5000  <span style="color:blue">#forecast时模拟的次数</span>  
>  0  <span style="color:blue">#poly文件，0表示使用默认（csepetas目录下的template.poly）</span>  

* 配置文件csep.in  
子程序etas8pcsep使用，由主程序csepetas.r自动生成。也可以手动编辑然后手动执行etas8pesep.  
> 'input.dat' <span style="color:blue"># 预处理过的地震目录,符合etas8pcsep程序读取格式</span>   
> \* <span style="color:blue">#数据读取格式默认</span>   
> 2540.33333333333 <span style="color:blue">#time interval拟合用地震目录的时间跨度（单位：天），相对于tmin，大于tmin+time interval的地震不被采用（待确认）</span>   
> 1000 <span style="color:blue">#number of events,程序中会自动修正</span>   
> 0 <span style="color:blue">#地震目录起始日期（单位：天），tmin</span>   
> 3.7 <span style="color:blue">#拟合用最小震级</span>   
> 0 <span style="color:blue">#tstart，地震目录的起始日期</span>   
> 4 <span style="color:blue">#多边形的点数，下面是多边形端点列表</span>   
> 97 21   
> 107 21   
> 107 43   
> 97 43   
>  
> 41 41 <span style="color:blue">#预测格点分辨率</span>   
> 0.55776 <span style="color:blue">#下面是8个参数，含义参考para文件</span>   
> 0.36593   
> 0.0038714   
> 1.2931   
> 1.0678   
> 0.00065075   
> 2.2232   
> 0.52254   
> 4 0.01 <span style="color:blue">#预测用最小震级，空间平滑的smooth半径</span>   
>    
> 1 <span style="color:blue">#是否优化</span>   

* 输出文件para
>    mu            A                    c                   alpha              p                    d                    q                 gamma  
  1.102091020   0.251475617         0.002645224         1.571743480         1.063653882         0.001225893         3.513085470         0.684979403  
  1.038364069   0.231934299         0.002747038         1.581977372         1.069609287         0.001197059         3.502670737         0.692325991  
  1.038699969   0.231101622         0.002753615         1.582012925         1.069955590         0.001195360         3.500824690         0.692399104  
  1.038589046   0.230908800         0.002755115         1.582025534         1.070035534         0.001195045         3.500489848         0.692426650  
  1.038568346   0.230855006         0.002755534         1.582027418         1.070058022         0.001194955         3.500397382         0.692436436  
  1.038565568   0.230839850         0.002755653         1.582027460         1.070064430         0.001194929         3.500371335         0.692439369  


## Dispatcher 文件解析
* self.__options = DispatcherOptionParser.DispatcherOptionParser().options(required)执行parse命令行option以及Forecast,ForecastHandlerFactory,CSEPRandom,EvaluationTest,Catalog,GeographicalRegions,CSEPStorage,CSEPLogging等的初始化工作；返回python optparse后产生的object，保存在Dispatcher类的__options中。
* runDate使用命令行传入的Date，testDate为runDate-waitingPeriod, startdate和enddate对onedaymodel无效。
* ForecastFactory实例化时，可以接受命令行指定的testlist,testinputs,model list,model inputs，也可以接受配置文件中的相应设置，如果命令行指定models，则配置文件中的models和modelinputs失效:  
```python
def __init__ (self, dir_path,                                               
                    post_process = None,                                    
                    test_list = None,                                       
                    test_inputs = None,                                     
                    model_list = None,                                      
                    models_inputs = None,                                   
                    post_process_inputs = None):                            
   """ Initialization for ForecastGroup class. Forecast models to be invoked
      can be specified through a command-line (input variable) or through   
      initialization file.                                                  
      Constructor method checks for existence of initialization file, and   
      extracts forecast group specific values if it exists to initialize    
      internal data members."""                                             
```
* 具体的ForecastFactory是在ForecastGroup中使用CSEPContainer实例化，该类接受的参数包括：一种factory类型（如ForecastFactory）、model列表、inputs(用CSEPInputParams.InputSeparator分割，对应model列表中的每一个model)，inputs是不同model初始化时的传入参数，传入顺序在common input之后。不同model对传入的参数有自己的处理方式。
* ETASModel接受的参数：optimization(0/1),randomSeedFile,historicalCatalog(0/1),parameterFile，我自己DIY了一个templateFile，用来接受加州以外地区作为计算区域。parse传入参数调用了CSEPInputParams.parse(dict,args,allow_extra_args=False)，其中dict表示默认参数列表，args为传入参数，allow_extra_args表示是否允许dict以外的参数传入。

## CSEPFACTORY
This class represents an interface for the object factory of registered Python
modules that are specific to the CSEP Natural Laboratory (NL).  通过解析config文件，load其中指定的modules（config文件中module元素）。其成员self.\_pythonModules是一个字典，其中的key是module的Type（CSEP中的类具有指定的Type属性，是一个字符串），在load之后，其value是python的class，可以通过object()方法来实例化。 
* init函数：会读config文件并加载其中指定的module，如果传入parent_class，那么config文件中的module必须是parent_class的子类 
* object函数：实例化一个类，传入参数class_type和input_variable
* keys函数：查看factory中所有类的type，实际上也就是查看self.\_pythonModules的key
* loadPythonModules函数：加载module，传入参数config_file和parent_class，init函数就是调用它来初始化。


## ETAS_DROneDay Model
* 处理zmap地震目录时，为什么要把12月的年份减1？(/usr/local/csep/src/SCECModels/SNZone/src/EEPAS/reformatscec.f)

## Test注意事项
* Evaluation使用RELMCatalog对地震目录进行处理，该类只能选取某个时间点以后的地震事件，某时间点之前的事件需要事先手动选出。
* Ntest的画图：
    1. RELMNumberTest.py中的plot函数，并会调用\_\_plotData，一个调用堆栈示例如下
>  /usr/local/csep/src/generic/EvaluationTest.py(1445)&lt;module&gt;()  
> -> each_test.resultData()  
>  /usr/local/csep/src/generic/RELMTest.py(1019)resultData()  
> -> if EvaluationTest.resultData(self) is False:  
>  /usr/local/csep/src/generic/EvaluationTest.py(946)resultData()  
> -> plots = self.\__class\__.plot(entry_path)  
> /usr/local/csep/src/generic/RELMNumberTest.py(426)plot()  

    画出的图模样如下，其中绿线是cdf，使用Poisson分布，以预测的发生率为参数。两条竖的直线，实线为观测到的地震数，虚线为预测地震数。
    灰色框标出97.5%和2.5%的位置（即：拒绝域的边缘）。![cdf图](pics/cdf_ntest.svg)
    2. plotAllModelsSummary函数，调用堆栈示例如下：
>  /usr/local/csep/src/generic/EvaluationTest.py(1445)&lt;module&gt;()  
-> each_test.resultData()  
  /usr/local/csep/src/generic/RELMTest.py(1019)resultData()  
-> if EvaluationTest.resultData(self) is False:  
  /usr/local/csep/src/generic/EvaluationTest.py(1020)resultData()  
-> test_obj = self)  
> /usr/local/csep/src/generic/RELMNumberTest.py(327)plotAllModelsSummary()  
-> plt.plot([event_count],  

    画出的图模样如下，其中方块为观测到的地震数，绿色表示接受，红色表示拒绝；小竖线为预测地震数，x轴的范围是2.5%到97.5%概率（即：拒绝域的边缘）。![summaryall图](pics/summary_all_ntest.png)
    3. plotSummary函数，调用堆栈示例如下：
    >   /usr/local/csep/src/generic/EvaluationTest.py(1445)&lt;module&gt;()  
-> each_test.resultData()  
  /usr/local/csep/src/generic/RELMTest.py(1019)resultData()  
-> if EvaluationTest.resultData(self) is False:  
  /usr/local/csep/src/generic/EvaluationTest.py(940)resultData()  
-> self.updateCumulativeResultData(entry_path)  
  /usr/local/csep/src/generic/EvaluationTest.py(555)updateCumulativeResultData()  
-> image_files = self.__class__.plotSummary(summary_file)  
 /usr/local/csep/src/generic/RELMTest.py(1201)plotSummary()  
    
    画的是delta1和delta2，定义如下： 
    ```python
      delta1=1 - poisscdf(number_quakes - epsilon, number_forecast_quakes)  
      delta2=poisscdf(number_quakes + epsilon, number_forecast_quakes)  
    ```
    其中Ntest的$epsilon=1.0\times 10^{-6}$
    ![summary_N](pics/summary_ntest.png)
* LTest的一些要点：
    * RELMLikelihoodTest.py:\_\_invoke()中log_likelihoods_simulations是模拟的log_likelihood和实际log_liklihood之差，也就是liklihood之比。
    * plot函数画的是log_likelihoods_simulations的cdf，方法是：先把log_likelihoods_simulations排序，作为横坐标；给它们编号然后归一化作为纵坐标。
    ![cdf_ltest](pics/cdf_ltest.png)
    * plotAllModelsSummary函数，画的是真实的log_likelihood在模拟的log_likelihood之间的位置，置信区间同NTest，这里直接选取模拟log_likelihood的counting分位点。
    ![summary_all_ltest](pics/summary_all_ltest.png)
    * plotSummary函数，画的是在多次的模拟中地震发生的比重，如果实际log_likelihood比模拟的大，则认为发生。
    ```python
    len(func(simulations,event_count)) / float(len(simulations))
    ```
    其中func返回simulations中小于event_count的index，event_count=0。而simulations是模拟的log_likelihood和实际log_liklihood之差。
    ![summary_ltest](pics/summary_ltest.png)
* \_simulationSingleRun的算法  
   函数位置：RELMTest.py，如果传入地震catalog，使用catalog，否则按如下算法生成：  
   1. 先由总的rate生成发生的地震数:
   ```python
   number_events = int(scipy.stats.poisson.ppf(generator.createNumbers(num)[0],
           total_forecast_rate))  
   ```                                                      
   其中generator.createNumbers(num)[0]是一个0~1之间的随机数（可以用seed固定），然后按ppf生成地震数。
   2. 依次判断每个地震在哪个bin中：
        把每个bin的大小按当中的rate放缩，然后把地震随机撒，看落在哪个bin中。程序中把每个bin的rate生成一个类似于cdf一样的函数，然后
        生成0~1之间的随机数，看落在cdf的哪个位置。
         ```python
         found_index = np.searchsorted(rates_CDF,
          random_numbers[index],
          side='right')   
          ```
   3. 计算logliklihood